# Creating Word Vectors with word2vec

In this notebook, we create word vectors from a corpus of IMDB movie reviews.

#### Set seed for reproducibility

In [1]:
import numpy as np
np.random.seed(42)

#### Load dependencies

In [2]:
import nltk


#### Load data

In [5]:
nltk.download() # show List; use Download, "brown", Quit

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> brown
      Unzipping corpora/brown.zip.

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

In [6]:
from nltk.corpus import brown

In [7]:
brown.words()

['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', ...]

In [8]:
len(brown.words())

1161192

In [9]:
brown.categories() # e.g., news from Chicago Tribune, reviews from Time Magazine; many others are books by an individual author

['adventure',
 'belles_lettres',
 'editorial',
 'fiction',
 'government',
 'hobbies',
 'humor',
 'learned',
 'lore',
 'mystery',
 'news',
 'religion',
 'reviews',
 'romance',
 'science_fiction']

In [11]:
for c in brown.categories():
    print(c, len(brown.words(categories=c)))

adventure 69342
belles_lettres 173096
editorial 61604
fiction 68488
government 70117
hobbies 82345
humor 21695
learned 181888
lore 110299
mystery 57169
news 100554
religion 39399
reviews 40704
romance 70022
science_fiction 14470
